## 추천시스템

### 학습 목표
- 추천 시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 추천 시스템이 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다.
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.

### 루브릭 
-  CSR matrix가 정상적으로 만들어졌다. : **사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.**
- MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. : **사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.**
- 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. : **MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.**

## 목차
### 1. 데이터 준비와 전처리
#### 1) 데이터 로드
#### 2) 데이터 분석
#### 3) 
### 2. 모델 구성
#### 1) CSR matrix
#### 2) 
#### 3)
### 3. 모델 평가
#### 1) 좋아하는 영화와 비슷한 영화
#### 2) 
---

## 1. 데이터 준비와 전처리
### 1) 데이터 로드

In [2]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 2) 데이터 분석
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [11]:
ratings['movie_id'].nunique()

3628

In [12]:
ratings['user_id'].nunique()

6039

In [14]:
ratings = pd.merge(left = ratings, right = movies, how = 'left', on = 'movie_id')
ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [18]:
movie_count = ratings.groupby('title')['counts'].sum()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   14449
Star Wars: Episode IV - A New Hope (1977)                13178
Star Wars: Episode V - The Empire Strikes Back (1980)    12648
Saving Private Ryan (1998)                               11348
Star Wars: Episode VI - Return of the Jedi (1983)        11303
Raiders of the Lost Ark (1981)                           11179
Silence of the Lambs, The (1991)                         11096
Matrix, The (1999)                                       10903
Sixth Sense, The (1999)                                  10703
Terminator 2: Judgment Day (1991)                        10513
Fargo (1996)                                             10465
Schindler's List (1993)                                  10317
Braveheart (1995)                                        10125
Shawshank Redemption, The (1994)                         10085
Back to the Future (1985)                                10081
Godfather, The (1972)                            